In [1]:
import gc
gc.enable()

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import h2o
from h2o.automl import H2OAutoML

SEED = 2024

In [2]:
h2o.init(max_mem_size="20G")
h2o.display.toggle_user_tips('off')

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.22" 2024-01-16; OpenJDK Runtime Environment (build 11.0.22+7-post-Ubuntu-0ubuntu220.04.1); OpenJDK 64-Bit Server VM (build 11.0.22+7-post-Ubuntu-0ubuntu220.04.1, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpdb2imp5n
  JVM stdout: /tmp/tmpdb2imp5n/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpdb2imp5n/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,1 month and 17 days
H2O_cluster_name:,H2O_from_python_unknownUser_wob0f6
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,20 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [3]:
DATA_DIR = '/kaggle/input/playground-series-s4e5'

train = pd.read_csv(f'{DATA_DIR}/train.csv')
test = pd.read_csv(f'{DATA_DIR}/test.csv')

In [4]:
train = train.drop('id', axis=1)
test = test.drop('id', axis=1)

In [5]:
TARGET = 'FloodProbability'
features = [f for f in test.columns]

In [6]:
time_limit = 60 * 60 * 11
aml = H2OAutoML(
    nfolds=10,
    max_runtime_secs=time_limit,
    stopping_metric='deviance',
    stopping_rounds=100,
    stopping_tolerance=1e-3,
    sort_metric='deviance',
    exploitation_ratio=0.1,
    seed=SEED)

aml.train(x=features, y=TARGET, training_frame=h2o.H2OFrame(train))

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_1_AutoML_1_20240501_41322


Status of Neuron Layers: predicting FloodProbability, regression, gaussian distribution, Quadratic loss, 441 weights/biases, 12.9 KB, 11,199,789 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight           weight_rms           mean_bias             bias_rms
--  -------  -------  ---------  ---------  ----  ----  ---------------------  ---------------------  ----------  --------------------  -------------------  --------------------  -----------------------
    1        20       Input      0.0
    2        10       Rectifier  0.0        0.0   0.0   0.0032280068351246884  0.0037750480696558952  0.0         0.12397275666589849   0.43003249168395996  -0.5422773793473897   1.8385944366455078
    3        10       Rectifier  0.0        0.0   0.0   0.007710912426118739   0.01268523558974266    0.0         -0.13141942511312663  0.9729926586151123   1.4408270031511896    0.633786678314209
    4        10       Rectifier  0.0        0.0   0.0   0.047468539957189936   0.07226830720901489    0.0         -0.43032555595738814  1.8489179611206055   1.7887080566043163    1.0994715690612793
    5        1        Linear                0.0   0.0   0.0018424549212795681  0.0018620104528963566  0.0         -0.08056849986314774  0.2723979949951172   -0.14604627392403427  1.0971281125650402e-154

ModelMetricsRegression: deeplearning
** Reported on train data. **

MSE: 0.00035526245496915465
RMSE: 0.01884840722631901
MAE: 0.014915361898105387
RMSLE: 0.012589796280532776
Mean Residual Deviance: 0.00035526245496915465

ModelMetricsRegression: deeplearning
** Reported on cross-validation data. **

MSE: 0.0003599679534505362
RMSE: 0.018972821441486667
MAE: 0.014904585421015866
RMSLE: 0.012684264651469001
Mean Residual Deviance: 0.0003599679534505362

Cross-Validation Metrics Summary: 
                        mean         sd           cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid    cv_6_valid    cv_7_valid    cv_8_valid    cv_9_valid    cv_10_valid
----------------------  -----------  -----------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  -------------
aic                     nan          0            nan           nan           nan           nan           nan           nan           nan           nan           nan           nan
loglikelihood           nan          0            nan           nan           nan           nan           nan           nan           nan           nan           nan           nan
mae                     0.0149046    0.000129411  0.0149265     0.0150027     0.0150801     0.0149882     0.014918      0.0149983     0.0149245     0.0146815     0.0148061     0.0147201
mean_residual_deviance  0.000359968  2.39511e-06  0.000358163   0.000360047   0.000364163   0.000361128   0.000360411   0.000361042   0.000360601   0.000354783   0.00035912    0.000360222
mse                     0.000359968  2.39511e-06  0.000358163   0.000360047   0.000364163   0.000361128   0.000360411   0.000361042   0.000360601   0.000354783   0.00035912    0.000360222
r2                      0.861741     0.00115554   0.863521      0.861678      0.860476      0.861321      0.861807      0.860972      0.860831      0.863494      0.862785      0.860524
residual_deviance       0.000359968  2.39511e-06  0.000358163   0.000360047   0.000364163   0.000361128   0.000360411   0.000361042   0.000360601   0.000354783   0.00035912    0.000360222
rmse                    0.0189727    6.31794e-05  0.0189252     0.0189749     0.0190831     0.0190034     0.0189845     0.0190011     0.0189895     0.0188357     0.0189505     0.0189795
rmsle                   0.0126842    4.47147e-05  0.012657      0.0126864     0.0127633     0.0127078     0.0126845     0

In [7]:
preds = aml.predict(h2o.H2OFrame(test[features]))

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%


In [8]:
sub = pd.read_csv(f'{DATA_DIR}/sample_submission.csv')
sub[TARGET] = preds.as_data_frame()
sub[TARGET] = sub[TARGET].clip(0, 1)

/opt/conda/lib/python3.10/site-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above) and activate it using:

with h2o.utils.threading.local_context(polars_enabled=True, datatable_enabled=True):
    pandas_df = h2o_df.as_data_frame()

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


In [9]:
sub.to_csv('nb04.csv', index=False)

In [10]:
!head nb04.csv

id,FloodProbability
1117957,0.5771843372359773
1117958,0.4489905961829521
1117959,0.449316687057833
1117960,0.4723005324531881
1117961,0.4732269256989082
1117962,0.5155771111369621
1117963,0.5369692363536936
1117964,0.5356465630320362
1117965,0.4740977182862526
